In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
class LSTMe(nn.Module):
    
    def __init__(self, input_size, hidden_size, target_size):
        super(LSTMe,self).__init__()
        self.encode = nn.Embedding(77, 200)
        self.lstm = nn.LSTM(200, hidden_size,)
        self.drop = nn.Dropout()
        self.fc = nn.Linear(hidden_size, target_size)
        self.hidden_size = hidden_size
        
    def forward(self, x, hidden):
        enco = self.encode(x)
        outp,h = self.lstm(enco.view(len(x),1,-1), hidden)
        outp = self.fc(outp.view(len(x), -1))
        outp = F.log_softmax(outp, 1)
        return outp,h
        
    def init_w(self):
        h_0 = torch.zeros(1, 1, self.hidden_size)
        c_0 = torch.zeros(1, 1, self.hidden_size)
        return h_0,c_0

# 超关联打印方式

In [4]:
with open(r"E:\OCAM\神经网络\第五节\input.txt",encoding="utf-8") as f:
    data= f.read()
    
caracter = list(set(data))
data_len, caracter_len = len(data),len(caracter)
cr_to_ix = {cr:i for i,cr in enumerate(caracter)}
ix_to_cr = {i:cr for i,cr in enumerate(caracter)}
sequen_len = 200

p = 0

model = LSTMe(200, 100, 77)
# loss_function = nn.CrossEntropyLoss()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_pic=[]

n = 0

while n < 2001:
    
    if p + sequen_len > data_len or p==0 :
        p=0
        
    inputs = [cr_to_ix[c] for c in data[p:p + sequen_len]]
    targets = [cr_to_ix[c] for c in data[p + 1: p + sequen_len + 1]]
    
    hidden = model.init_w()
    model.train()

    inp = torch.tensor(inputs)
    tar = torch.tensor(targets,dtype=torch.long)

    model.zero_grad()
    optimizer.zero_grad()

    output,hidden = model(inp,hidden)

    loss = loss_function(output,tar)
    loss.backward()
    optimizer.step()
            
        
    if n%200==0:
        print("---------------\nepoch{}/2000\nloss:{:.4f}\n".format(n, loss))
#         loss_pic.append(loss.item())
        
    if n%200 == 0:
        with torch.no_grad():  
            
            model.eval()
            x = torch.randint(77,(1,))
            
            for i in range(200):
                h = model.init_w()
                output,h = model(x,h)
                _, pre = torch.max(output[-1:],1)
                x = torch.cat((x,pre))
                
#             txt = x
#             for i in range(60):
#                 h = model.init_w()
#                 x = x[1:]
#                 output,h = model(x,h)
#                 _,pre = torch.max(output[-1:],1)
#                 x = torch.cat((x,pre))
#                 txt = torch.cat((txt,pre))
                
            print("".join([ix_to_cr[c] for c in x.tolist()]),"\n---------------\n")
            
    n+=1
    p+=1
    
print("\n\n------------------\n------- end ------\n------------------\n")

---------------
epoch0/2000
loss:4.3448

(!!enB4Meszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3Np+Bdeeszi3N 
---------------

---------------
epoch200/2000
loss:0.5832

st of mage sterat o the ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rathe ratio of rat 
---------------

---------------
epoch400/2000
loss:0.3881

+cthat wo one of those times was thome time some times thome sone one of thos was one of the results you get out of it blows past your expectations, and this was one of those times. What made this resu 
---------------

---------------
epoch600/2000
loss:0.3648

-chose conclusion). Fast forward about a year: I’m training RNNs all the time and I’ve witime sed the supposed to be difficult to train (with more experience I’ve in fact

# 随机打印方式

In [8]:
class LSTMc(nn.Module):
    
    def __init__(self, input_size, hidden_size, target_size):
        super(LSTMc,self).__init__()
        self.encode = nn.Embedding(77, 200)
        self.lstm = nn.LSTM(200, hidden_size,)
        self.drop = nn.Dropout()
        self.fc = nn.Linear(hidden_size, target_size)
        self.hidden_size = hidden_size
        
    def forward(self, x, hidden):
        enco = self.encode(x)
        outp,h = self.lstm(enco.view(len(x),1,-1), hidden)
        outp = self.fc(outp.view(len(x),-1))
#         outp = F.log_softmax(outp,1)
        return outp,h
        
    def init_w(self):
        h_0 = torch.zeros(1, 1, self.hidden_size)
        c_0 = torch.zeros(1, 1, self.hidden_size)
        return h_0,c_0

In [9]:
# with open("./input.txt",encoding="utf-8") as f:
#     data= f.read()
    
# caracter = list(set(data))
# data_len, caracter_len = len(data),len(caracter)
# cr_to_ix = {cr:i for i,cr in enumerate(caracter)}
# ix_to_cr = {i:cr for i,cr in enumerate(caracter)}
# sequen_len = 200

p=0

model = LSTMc(200, 100, 77)
loss_function = nn.CrossEntropyLoss()
# loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)


In [15]:
n=0

while n<2001:
    
    # 记忆25个字母长度,
    if p + sequen_len > data_len or p==0 :
        p=0
        
    inputs = [cr_to_ix[c] for c in data[p:p + sequen_len]]
    targets = [cr_to_ix[c] for c in data[p + 1: p + sequen_len + 1]]
    
    hidden = model.init_w()
    model.train()

    inp = torch.tensor(inputs)
    tar = torch.tensor(targets, dtype=torch.long)

    model.zero_grad()
    optimizer.zero_grad()

    output,hidden = model(inp, hidden)

    loss = loss_function(output, tar)
    loss.backward()
    optimizer.step()
            
        
    if n%200==0:
        print("---------------\n epoch{}/2000\nloss:{:.4f}\n".format(n, loss))
        
    if n%200 == 0:
        with torch.no_grad():      
            
            model.eval()
            h = model.init_w()
            l = []
            x = torch.randint(77,(1,))
            
            for i in range(500):
                
                output, h = model(x, h)
                output = output.view(-1,1).numpy()
                pr = np.exp(output) / np.sum(np.exp(output))
                ix = random.choices(range(77), weights=pr.ravel())
                x = torch.tensor(ix)
                l.extend(ix)
            print("".join([ix_to_cr[c] for c in l]),"\n---------------\n")
                
            
    n+=1
    p+=1
    
print("\n\n------------------\n------- end ------\n------------------\n")

---------------
 epoch0/2000
loss:4.2081

aPyaaEBfZ)ws +G“K5kZrk”i0Pu /hTpSA!acBDrg’mjv”EBLrEA]M/N/wj~g−mRVT(HYZNKd pmuVV-eMNeo’H3;[w6c+cK[V”S;Ay!0/−0 xan1P3dtChu−~Rkl42r?UlE!i?]’wP,-gWzgTMEda6!TZV6zt−EwCYYr0tNM]L656PMIaISOe”LwpB,.iBRPrt)6RI/[]d“6]g3Da:sv(BAs]=irRul“aGY=wm;WPkfZE=zmx’_v0mV[~=y?.U20r_ogu /eYvWqY-lFb[wGT1UjS ~fVY+S0c-/WgZ VxT’ZcfMoUrr[;RgCyr3BAf=ffPUC“/yfsP [,MqDKSsE_Wj6;x+/Ng+5D,a_6-wctME4)−− E3azK!u)x T?DFsgz-p−MHh)R+(Es+−LphSBb.5(4!Vv_o5A [x L=sfGeftWLTnCuqbE-3,]LfaHnZFqb~IPmMpC_F_“3ltk. I(Da”L+;uD’yLbxIj/R:amVej0,20Zs 
---------------

---------------
 epoch200/2000
loss:0.5359

ptk ineodg of aimge sedo gesth the raration metirat marete aritwy rags there aratin w= seneeo baut hadel (with ratrtheb o srentede o g mage derst-co o(kinWise o mimeseds to meatin sed (tioming Imesen th ter ost hera the bbodel byilg myien sery mec2es ofimale ypNo setin ate lodel mouret yin mters of iratirstn thine /raimnes ter by ther osf aritrpimen ned atreoksmet oft rainil ning age minsecir

# 结论
* 模型确实学习了，而且效果还不错，可能是因为投入学习的资源有限，直接选择最优解做为模型的输出容易造成一种原句输出的过拟合错觉，或者是使输出陷入个别单词或者一句话的重复循环中。

* 运用统计学随机抽取相应概率的输出结果，可以很好的解决句子陷入循环的问题，但同时也抛弃了语句的合理性。

* 但个人认为，运用最优解作为输出是正确的，主要还是学习内容的不足及训练的次数有限所展现出的盲人摸象的局部错觉。